### Demonstration of GPU Accelerated SigMF Reader

Please note that our work with both SigMF readers and writers is focused on appropriately handling the data payload on GPU. This is similar to our usage of DPDK within the Aerial SDK and cuVNF.

In [1]:
import json
import numpy as np
import cupy as cp
import cusignal

We are using the [Northeastern University Oracle RF Fingerprinting Dataset](http://www.genesys-lab.org/oracle):

In [2]:
# set input SigMF dataset - note that this is ~28gb
base_url = 'https://repository.library.northeastern.edu/downloads/neu:m044q5210?datastream_id=content'
path = 'KRI-16Devices-RawData'
file = '2ft/WiFi_air_X310_3123D7B_2ft_run1'

# uncomment to use smaller ~300mb dataset
# base_url = 'https://repository.library.northeastern.edu/downloads/neu:m044q523j?datastream_id=content'
# path = 'KRI-16IQImbalances-DemodulatedData'
# file = 'Demod_WiFi_cable_X310_3123D76_IQ#1_run1'

comp = '.zip'

In [3]:
import os

# create data directory
data_dir = '../data/'
if not os.path.exists(data_dir):
    print('creating data directory')
    os.system('mkdir ../data')

# download dataset
if not os.path.isdir(data_dir + path):
    if not os.path.isfile(data_dir + path + comp):
        print(f'Downloading {base_url} to {data_dir + path + comp}')
        os.system(f'curl -o {data_dir + path + comp} {base_url}')
    print(f'Decompressing {data_dir + path + comp}...')
    os.system(f'unzip {data_dir + path + comp} -d {data_dir}')
    print(f'{data_dir + path + comp} decompressed')
else:
    print(f'Dataset already exists under {data_dir + path}')
    
# set input SigMF files
meta_file = os.path.join(data_dir, path, file + '.sigmf-meta')
data_file = os.path.join(data_dir, path, file + '.sigmf-data')

Dataset already exists under ../data/KRI-16Devices-RawData


# Reader (Binary and SigMF)

For our purposes here, [SigMF](https://github.com/gnuradio/SigMF) data is treated as a JSON header and processed on CPU, while the *binary* payload file is mapped to GPU and cuSignal uses a CUDA kernel to parse the file. While we've focused on SigMF here, you can use the underlying `cusignal.read_bin` and `cusignal.parse_bin` (and corresponding write functions) for your own datasets.

### Baseline Reader (CPU, Numpy)

In [4]:
with open(meta_file, 'r') as f:
    md = json.loads(f.read())

if md['_metadata']['global']['core:datatype'] == 'cf32':
    data_type = np.complex64

In [5]:
%%timeit
data_cpu = np.fromfile(data_file, dtype=data_type)

134 ms ± 483 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Baseline Reader (GPU, Numpy)

In [6]:
%%timeit
data_gpu = cp.fromfile(data_file, dtype=data_type)
cp.cuda.runtime.deviceSynchronize()

194 ms ± 241 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### cuSignal - Use Paged Memory (Default)

This method is preferred for offline signal processing and is the easiest to use

In [7]:
%%timeit
data_cusignal = cusignal.read_sigmf(data_file, meta_file)
cp.cuda.runtime.deviceSynchronize()

98.7 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### cuSignal - Use Pinned Buffer (Pinned)

This method is preferred for online signal processing tasks when you're streaming data to the GPU with known and consistent data sizes

In [8]:
binary = cusignal.read_bin(data_file)
buffer = cusignal.get_pinned_mem(binary.shape, cp.ubyte)

In [9]:
%%timeit
data_cusignal_pinned = cusignal.read_sigmf(data_file, meta_file, buffer)
cp.cuda.runtime.deviceSynchronize()

ValueError: could not broadcast input array from shape (40012800,) into shape (320102400,)

### cuSignal - Use Shared Buffer (Mapped)

This method is preferred for the Jetson line of embedded GPUs. We're showing performance here on a PCIe GPU (which is why it's so slow!)

In [10]:
binary = cusignal.read_bin(data_file)
buffer = cusignal.get_shared_mem(binary.shape, cp.ubyte)

In [11]:
%%timeit
data_cusignal_shared = cusignal.read_sigmf(data_file, meta_file, buffer)
cp.cuda.runtime.deviceSynchronize()

ValueError: could not broadcast input array from shape (40012800,) into shape (320102400,)

# Writer (Binary and SigMF)

In [12]:
import os

sigmf = cusignal.read_sigmf(data_file, meta_file)
test_file_ext = "test-data.sigmf-data"

if os.path.exists(test_file_ext):
    os.remove(test_file_ext)

### Baseline Writer

In [13]:
%%timeit
sigmf.tofile(test_file_ext)
cp.cuda.runtime.deviceSynchronize()

633 ms ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### cuSignal - Use Paged Memory (Default)

In [14]:
%%timeit
cusignal.write_sigmf(test_file_ext, sigmf, append=False)
cp.cuda.runtime.deviceSynchronize()

632 ms ± 271 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### cuSignal - Use Pinned Buffer (Pinned)

In [15]:
binary = cusignal.read_bin(data_file)
buffer = cusignal.get_pinned_mem(binary.shape, cp.ubyte)

In [16]:
%%timeit
cusignal.write_sigmf(test_file_ext, sigmf, buffer, append=False)
cp.cuda.runtime.deviceSynchronize()

557 ms ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### cuSignal - Use Mapped Buffer (Mapped)

In [17]:
binary = cusignal.read_bin(data_file)
buffer = cusignal.get_shared_mem(binary.shape, cp.ubyte)

In [18]:
%%timeit
cusignal.write_sigmf(test_file_ext, sigmf, buffer, append=False)
cp.cuda.runtime.deviceSynchronize()

557 ms ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
